In [1]:
from libs.plugin_converter.semantic_kernel_v0_to_openai_function import generate_callables, generate_definitions, make_context
from libs.plugins.plugin_capital import PluginCapital
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool

import os
from libs.utils.connector_llm import *
from libs.plugins.plugin_capital import PluginCapital
import libs.plugin_converter.semantic_kernel_v0_to_openai_function as semantic_kernel_v0_to_openai_function
import libs.plugin_converter.openai_function_to_tool as openai_function_to_tool
from libs.plugin_orchestrator.implementation_tool import OrchestratorWithTool
from libs.plugin_orchestrator.implementation_bare import OrchestratorBare

import dotenv
dotenv.load_dotenv()

True

In [2]:
#llm = factory_create_connector_llm(
#    provider='llama_cpp',
#    modelname='not-needed',
#    version='not-needed',
#    credentials=CredentialsOpenAI(
#        base_url='http://llm-server:8080/v1',
#        api_key='not-needed',
#    )
#)

llm = factory_create_connector_llm(
    provider='azure_openai',
    modelname=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT"),
    version=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME_CHAT").split('-')[-1],
    credentials=CredentialsOpenAI(
        base_url=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_KEY"),
    ),
    hyperparameters={'tool_choice': 'none'}
)

orchestrator = OrchestratorBare(
    connection = llm,
    tool_definitions = openai_function_to_tool.generate_definitions(semantic_kernel_v0_to_openai_function.generate_definitions([(PluginCapital(), "PluginCapital")])),
    tool_callables = generate_callables([(PluginCapital(), "PluginCapital")]),
    token_limit_input = 1024,
    token_limit_output = None,
    max_steps_recommended = 2,
    max_steps_allowed = 3,
    prompt_app_system='You are an AI assistant whose goal is to answer the user question.',
    prompt_app_user='What is the capital of france?',
)

r = await orchestrator.run()
print(r)
print('\n\n' + '-'*30 + '\n\n')
assert type(r.answer) == str
assert len(r.answer) > 4
print(r.answer)

THIS IS WHAT THE LLM IS TRYING TO DO:
content='{"thought": "I need to retrieve the capital of France.", "action_name": "PluginCapital_get_capital", "args": {"country": "France"}}' role='assistant' tool_calls=None
[2024-08-12 14:04:10 +0000] libs.libs.plugin_orchestrator INFO     Function calling detected by: custom implementation
[2024-08-12 14:04:10 +0000] libs.libs.plugin_orchestrator INFO     At step 1 the function PluginCapital_get_capital was requested, with args = {'country': 'France'}
[2024-08-12 14:04:10 +0000] libs.libs.plugin_orchestrator INFO     result: Executed function PluginCapital_get_capital and returned: paris
As this was the step 1/2, next you should call 'answer' with all the results you have.
THIS IS WHAT THE LLM IS TRYING TO DO:
content='{"thought": "The task is complete, I have obtained the information about the capital of France.", "action_name": "answer", "args": {"text": "The capital of France is <strong>Paris</strong>."}}' role='assistant' tool_calls=None
[20

In [4]:
orchestrator._full_message_history_for_debugging

[ChatCompletionMessage(role='system', content='# General instructions\nYour goal is to answer the user question, and you can call functions (aka commands/actions) to achieve that.\nYour decisions must always be made independently without seeking user assistance.\nEvery action has a cost, so be smart and efficient. You\'ll be rewarded if you answer correctly and quickly.\nAim to complete the goal in the least number of steps (aka function calls). Whenever you are ready to answer, use the function \'answer\'.\nDo not take more than 2 steps to complete a task (including the function \'answer\').\nIf at step 1 you don\'t have all necessary information, answer the best you can with the information at hand.\nAlways try to answer with as much useful information as you can.\n\nYou are allowed to call the following function:\n1. PluginCapital_get_capital: Returns the name of the capital of a country..\n  Arguments: country (Name of the country. Type: string. Required: yes)\n2. answer: Send resp